### Scryfall API Setup

In [1]:
SF_API_URL = 'https://api.scryfall.com'
SF_API_HEADERS = {
    'User_Agent' : 'card-classifier/0.1',
    'Accept' : '*/*'
}
SF_API_DELAY = 50 # ms - No ratelimit on *.scryfall.io

In [2]:
import requests
import json
import os

### Pull all download URIs

response = requests.get(SF_API_URL+'/bulk-data', headers=SF_API_HEADERS)

data = response.json()['data']
# print(json.dumps(data, indent=4))

# Extract all URI
download_uri = {}
for object in data:
    download_uri[object['type']] = object['download_uri']
print("URI paths saved")

# Download and save the unique artwork file
artwork_file_path = os.path.join('data', 'unique_artwork.json')
response = requests.get(download_uri['unique_artwork'], headers=SF_API_HEADERS)
with open(artwork_file_path, 'wb') as file:
    file.write(response.content)

URI paths saved


### Save Card Images

In [3]:
with open(artwork_file_path, 'r', encoding="utf8") as file:
    unique_artwork_data = json.load(file)

In [4]:
# print(json.dumps(unique_artwork_data[4], indent=4))

In [ ]:
from concurrent.futures import ThreadPoolExecutor

def download_image(entry, is_dfc=False, card_face=None):
    if not is_dfc:
        filename = f"{entry['name'].replace('/', '_')}_{entry['set']}_{entry['collector_number']}_{entry['id']}.jpg"
        art_path = os.path.join('data/art', filename)
        image_url = entry['image_uris']['art_crop']
    else:
        filename = f"{card_face['name'].replace('/', '_')}_{entry['set']}_{entry['collector_number']}_{entry['id']}.jpg"
        art_path = os.path.join('data/art', filename)
        image_url = card_face['image_uris']['art_crop']

    response = requests.get(image_url, headers=SF_API_HEADERS)
    if response.status_code == 200:
        with open(art_path, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded: {filename}")
    else:
        print(f"Failed to download: {filename} (Status code: {response.status_code})")

# Create a thread pool
with ThreadPoolExecutor() as executor:
    futures = []
    for art_entry in unique_artwork_data:
        # If not DFC
        if 'image_uris' in art_entry:
            futures.append(executor.submit(download_image, art_entry))
        # If DFC
        else:
            for card_face in art_entry['card_faces']:
                futures.append(executor.submit(download_image, art_entry, True, card_face))


Downloaded: Fell Beast's Shriek_altc_18_000225fc-9bc3-4eb3-905e-02c19c873b0b.jpg
Downloaded: Forest_blb_280_0000419b-0bba-4488-8f7a-6194544ce91e.jpg
Downloaded: Siren Lookout_xln_78_0000cd57-91fe-411f-b798-646e965eec37.jpg
Downloaded: Essence Warden_prm_95481_000a7263-37e6-4246-82f9-94459517a5cc.jpg
Downloaded: Obyra's Attendants __ Desperate Parry_woe_63_0001e77a-7fff-49d2-a55c-42f6fdf6db08.jpg
Downloaded: Kor Outfitter_zen_21_00006596-1166-4a79-8443-ca9f82e6db4e.jpg
Downloaded: Fell Beast's Shriek_altc_18_000225fc-9bc3-4eb3-905e-02c19c873b0b.jpg
Downloaded: Fury Sliver_tsp_157_0000579f-7b35-4ed3-b44c-db2a538066fe.jpg
Downloaded: Ragged Playmate_dsk_150_000499a4-3f39-4d25-a2a2-b2f014e30754.jpg
Downloaded: Whiptongue Hydra_c18_36_0005c844-787c-4f0c-8d25-85cec151642b.jpg
Downloaded: Wall of Vipers_pcy_80_00042443-4d4e-4087-b4e5-5e781e7cc5fa.jpg
Downloaded: Venerable Knight_eld_35_0001f1ef-b957-4a55-b47f-14839cdbab6f.jpg
Downloaded: Wildcall_ugin_146_00020b05-ecb9-4603-8cc1-8cfa7a14befc.